In [6]:
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np

%matplotlib inline

## Input functions for the estimater

In [41]:
def parse(serialized):
    # Define a dict with the schema reflecting the data in the TFRecords file
    features = \
        {
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        }
    
    # Parse the serialized data
    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)
    
    # Get the image as raw bytes
    image_raw = parsed_example['image']
    
    # Convert the raw bytes to tensorflow datatypes
    image = tf.decode_raw(image_raw, tf.uint8)
    image = tf.cast(image, tf.float32)
    
    # Get the label
    label = parsed_example['label']
    
    # Return the image and label as correct data types
    return image, label
    

In [42]:
def input_fn(filenames, train, batch_size=32, buffer_size=2048):
    # Create a TensorFlow Dataset-object which has functionality for reading and shuffling data 
    # from TFREcords files
    
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    
    # Start building the pipeline
    # Parse
    dataset = dataset.map(parse)
    
    # Shuffle when training
    if train:
        dataset = dataset.shuffle(buffer_size = buffer_size)
        # Allow infinite reading of the data
        num_repeat = None
    else:
        num_repeat = 1
        
    # Repeat the dataset the given number of times
    dataset = dataset.repeat(num_repeat)
    
    # Set the batch size
    dataset = dataset.batch(batch_size)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    images_batch, labels_batch = iterator.get_next()
    
    return {'image':images_batch}, labels_batch
    
    

In [55]:
path_tfrecords_train = '../data/tfrecords/training.tfrecords'
def train_input_fn():
    return input_fn(filenames=path_tfrecords_train, train=True)

In [56]:
path_tfrecords_test = '../data/tfrecords/validation.tfrecords'
def test_input_fn():
    return input_fn(filenames=path_tfrecords_test, train=False)

In [57]:
image1 = np.array(Image.open('../data/validation/st1179_sound_9.PNG'))
image2 = np.array(Image.open('../data/validation/st1179_dry_knot_3.PNG'))
some_images = np.array([image1, image2])/255
some_images = np.reshape(some_images, (2, 37632))


In [58]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": some_images.astype(np.float32)},
    num_epochs=1,
    shuffle=False)

In [59]:
def model_fn(features, labels, mode, params):
    # Args:
    #
    # features: This is the x-arg from the input_fn.
    # labels:   This is the y-arg from the input_fn.
    # mode:     Either TRAIN, EVAL, or PREDICT
    # params:   User-defined hyper-parameters, e.g. learning-rate.
    
    # Reference to the tensor named "image" in the input-function.
    x = features["image"]

    # The convolutional layers expect 4-rank tensors
    # but x is a 2-rank tensor, so reshape it.
    net = tf.reshape(x, [-1, 112, 112, 3])    

    # First convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv1',
                           filters=32, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

    # Second convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv2',
                           filters=32, kernel_size=3,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)    

    # Flatten to a 2-rank tensor.
    net = tf.contrib.layers.flatten(net)
    # Eventually this should be replaced with:
    # net = tf.layers.flatten(net)

    # First fully-connected / dense layer.
    # This uses the ReLU activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc1',
                          units=128, activation=tf.nn.relu)    

    # Second fully-connected / dense layer.
    # This is the last layer so it does not use an activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc_2',
                          units=7)

    # Logits output of the neural network.
    logits = net

    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If the estimator is supposed to be in prediction-mode
        # then use the predicted class-number that is output by
        # the neural network. Optimization etc. is not needed.
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        # Otherwise the estimator is supposed to be in either
        # training or evaluation-mode. Note that the loss-function
        # is also required in Evaluation mode.
        
        # Define the loss-function to be optimized, by first
        # calculating the cross-entropy between the output of
        # the neural network and the true labels for the input data.
        # This gives the cross-entropy for each image in the batch.
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)

        # Reduce the cross-entropy batch-tensor to a single number
        # which can be used in optimization of the neural network.
        loss = tf.reduce_mean(cross_entropy)

        # Define the optimizer for improving the neural network.
        optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])

        # Get the TensorFlow op for doing a single optimization step.
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())

        # Define the evaluation metrics,
        # in this case the classification accuracy.
        metrics = \
        {
            "accuracy": tf.metrics.accuracy(labels, y_pred_cls)
        }

        # Wrap all of this in an EstimatorSpec.
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)
        
    return spec

In [60]:
params = {"learning_rate": 1e-4}

In [61]:
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,
                               model_dir="../checkpoints")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc628045240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [70]:
model.train(input_fn=train_input_fn, steps=8000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../checkpoints/model.ckpt-1400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1401 into ../checkpoints/model.ckpt.
INFO:tensorflow:loss = 1.3754535, step = 1401
INFO:tensorflow:global_step/sec: 46.1134
INFO:tensorflow:loss = 0.8299999, step = 1501 (2.170 sec)
INFO:tensorflow:global_step/sec: 49.3816
INFO:tensorflow:loss = 1.1330652, step = 1601 (2.025 sec)
INFO:tensorflow:global_step/sec: 51.4383
INFO:tensorflow:loss = 0.31214523, step = 1701 (1.946 sec)
INFO:tensorflow:global_step/sec: 49.7659
INFO:tensorflow:loss = 0.12395956, step = 1801 (2.009 sec)
INFO:tensorflow:global_step/sec: 50.5562
INFO:tensorflow:loss = 0.92764336, step = 1901 (1.976 sec)
INFO:tensorflow:global_step/sec: 50.4679
INFO:tensorflow:loss = 0.7987

In [71]:
result = model.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-12-03:33:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../checkpoints/model.ckpt-9400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-12-03:33:23
INFO:tensorflow:Saving dict for global step 9400: accuracy = 0.61051023, global_step = 9400, loss = 1.8392712


In [66]:
result

{'accuracy': 0.36923468, 'global_step': 1400, 'loss': 3.276199}

In [67]:
predictions = model.predict(input_fn=predict_input_fn)

In [69]:
cls_pred = np.array(list(predictions))
cls_pred

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../checkpoints/model.ckpt-1400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


array([3, 3])